In [5]:
import json, requests

r = requests.get('https://projects.fivethirtyeight.com/polls/polls.json')

polls = json.loads(r.content)

df = pd.DataFrame(polls)

def parse_answers(x):
    a = x['answers']
    idn = x['id']
    d = DataFrame(a)
    d['id'] = idn
    return d

dat = pd.concat(df[['answers', 'id']].apply(parse_answers, axis=1).values, sort=False)

df = df.iloc[:,1:].merge(dat, how='outer', on='id').copy()

df['pct'] = df['pct'].astype(float)
df['sampleSize'] = pd.to_numeric(df['sampleSize'])
df['startDate'] = pd.to_datetime(df['startDate'])
df['endDate'] = pd.to_datetime(df['endDate'])
df['created_at'] = pd.to_datetime(df['created_at'])

In [8]:
house = df.query('type == "house"').copy()
house['election'] = house['state'] + '-' + house['district'].astype(str)
len(house[['district', 'state']].drop_duplicates())
house['sampleSize'].describe()

count   1585.0000
mean     485.3009
std      189.5194
min       72.0000
25%      400.0000
50%      495.0000
75%      525.0000
max     2079.0000
Name: sampleSize, dtype: float64

In [10]:
polls = house.query('party in ["Dem", "Rep"]').sort_values('party').groupby(['state','id', 'district','created_at', 'sampleSize']).apply(lambda x: x['pct'].iloc[1] / x['pct'].sum() if len(x) == 2 else np.nan).reset_index()

In [12]:
polls.rename(columns={0:'rep_score', 'district': 'fivethirtyeight_district'}).to_feather('polls.feather')